# Character-level recurrent sequence-to-sequence model

**Author:** [fchollet](https://twitter.com/fchollet)<br>
**Date created:** 2017/09/29<br>
**Last modified:** 2020/04/26<br>
**Description:** Character-level recurrent sequence-to-sequence model.

## Introduction

This example demonstrates how to implement a basic character-level
recurrent sequence-to-sequence model. We apply it to translating
short English sentences into short French sentences,
character-by-character. Note that it is fairly unusual to
do character-level machine translation, as word-level
models are more common in this domain.

**Summary of the algorithm**

- We start with input sequences from a domain (e.g. English sentences)
    and corresponding target sequences from another domain
    (e.g. French sentences).
- An encoder LSTM turns input sequences to 2 state vectors
    (we keep the last LSTM state and discard the outputs).
- A decoder LSTM is trained to turn the target sequences into
    the same sequence but offset by one timestep in the future,
    a training process called "teacher forcing" in this context.
    It uses as initial state the state vectors from the encoder.
    Effectively, the decoder learns to generate `targets[t+1...]`
    given `targets[...t]`, conditioned on the input sequence.
- In inference mode, when we want to decode unknown input sequences, we:
    - Encode the input sequence into state vectors
    - Start with a target sequence of size 1
        (just the start-of-sequence character)
    - Feed the state vectors and 1-char target sequence
        to the decoder to produce predictions for the next character
    - Sample the next character using these predictions
        (we simply use argmax).
    - Append the sampled character to the target sequence
    - Repeat until we generate the end-of-sequence character or we
        hit the character limit.


In [ ]:
!pip install aicrowd-cli
%load_ext aicrowd.magic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%aicrowd login

Please login here: https://api.aicrowd.com/auth/p5zGic61w3C8FAMqlWzYHqf57ZhEwiqsx8922wIJR6M
API Key valid
Gitlab access token valid
Saved details successfully!


## Setup


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [ ]:
!pip install pywer
import pywer
import pandas as pd
import numpy as np
import os

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Download the data


In [ ]:
# !!curl -O http://www.manythings.org/anki/fra-eng.zip
# !!unzip fra-eng.zip


In [ ]:
!rm -rf data
!mkdir data
%aicrowd ds dl -c htrec-2022 -o data

test.csv:   0%|          | 0.00/45.5k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/395k [00:00<?, ?B/s]

In [ ]:
# with open(data_path, "r", encoding="utf-8") as f:
#     lines = f.read().split("\n")

In [ ]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

In [ ]:
import re
def clear_vowels(df,cols):
  for i in cols:
    df[i] = df[i].apply(lambda x: re.sub(r'ά|α|ὰ|ά|ᾄ|ᾅ|ὰ|ά|ᾳ|ᾴ|ᾶ|ᾷ|ἀ|ἁ|ἂ|ἃ|ἄ|ἅ|ἇ|Ἀ|Ἁ|Ἄ|Ἅ|Α','α',x.lower()))
    df[i] = df[i].apply(lambda x: re.sub(r'έ|ε|ὲ|έ|ἐ|ἑ|ἔ|ἕ|Ἐ|Ἑ|Ἔ|Ἕ|Ε','ε',x))
    df[i] = df[i].apply(lambda x: re.sub(r'ή|ή|η|ᾑ|ᾔ|ᾗ|ὴ|ή|ῃ|ῄ|ῆ|ῇ|ἠ|ἡ|ἢ|ἣ|ἤ|ἥ|ἦ|ἧ|Ἡ|Η','η',x))
    df[i] = df[i].apply(lambda x: re.sub(r'ΐ|ί|ι|ϊ|ὶ|ί|ῒ|ῖ|ἰ|ἱ|ἳ|ἴ|ἵ|ἶ|ἷ|Ἰ|Ἱ|Ἴ','ι',x))
    df[i] = df[i].apply(lambda x: re.sub( r'ο|ό|ὸ|ό|ὀ|ὁ|ὃ|ὄ|ὅ|Ὁ|Ὃ|Ὅ|Ο','ο',x))
    df[i] = df[i].apply(lambda x: re.sub(r'ῤ|ῥ','ρ',x))
    df[i] = df[i].apply(lambda x: re.sub(r'ῦ|υ|ύ|ὺ|ύ|ὐ|ὑ|ὓ|ὔ|ὕ|ὖ|ὗ|Ὑ','υ',x))
    df[i] = df[i].apply(lambda x: re.sub(r'ω|ᾠ|ᾤ|ᾦ|ᾧ|ὼ|ώ|ὠ|ὡ|ὢ|ὣ|ὤ|ὥ|ὦ|ὧ|Ὠ|Ὡ|Ὥ|Ὦ|ῳ|ῴ|ῶ|ῷ|ώ|Ω','ω',x))
    df[i] = df[i].apply(lambda x: re.sub(r'\W',' ',x)) #remove non words characters
    df[i] = df[i].apply(lambda x: re.sub(r'\s\s+',' ',x)) #collapse multiple whitespaces
  return(df)

clear_vowels(train_df,["HUMAN_TRANSCRIPTION","SYSTEM_TRANSCRIPTION"])

,HUMAN_TRANSCRIPTION,SYSTEM_TRANSCRIPTION,CENTURY,IMAGE_PATH,TEXT_LINE_NUM
0,εγγινομενα παθη μη σβεννυντες αλλα τη εκλυσει,εγγενομεναπαδημησμεννωτες αλλατηε κλησει,11,1 Bodleian-Library-MS-Barocci-102_00157_fol-75...,1
1,του βιου του καθ εαυτους πολλα γινεσθαι συγχωρουν,του β ου του καλεαυτους πολλαγινεσθαι συγχωρ ον,11,1 Bodleian-Library-MS-Barocci-102_00157_fol-75...,2
2,τες εμπυριζουσι τον αμπελωνα αλλα και ο δια,τες εμπυριζου σιμαμπελωνα αλλακαι οδξα,11,1 Bodleian-Library-MS-Barocci-102_00157_fol-75...,3
3,της ηδειας πλεονεξιας πολλους εις την των αλλ,της εδιας πλσον εξιας πολλους εις την των αλ,11,1 Bodleian-Library-MS-Barocci-102_00157_fol-75...,4
4,οτριων επιθυμιαν προκαλουμενος εμπυρι,λοτρλων επιθυμιαν προκαλουμενος εμπυρι,11,1 Bodleian-Library-MS-Barocci-102_00157_fol-75...,5
...,...,...,...,...,...
1870,σριαν του των ανων γενους γεγονε ανος και τι λε,σριαν τουτων ανων γενθυγεγονενανος οτιμε,10,100 Bodleian-Library-MS-Barocci-184_00040_fol-...,25
1871,γω την ημετεραν υπεδυ σαρκα και τ αλλα παν,γω την η μετεραν υπεθσαρκα ειταλλαπαν,10,100 Bodleian-Library-MS-Barocci-184_00040_fol-...,26
1872,τα ανινα υπεμεινεν οπου γε και τον στρον κατε,τα ανινα υπεμειμεν οπουγεκαι ετριν κατε,10,100 Bodleian-Library-MS-Barocci-184_00040_fol-...,27
1873,δεξατο ινα ημας τους υπο αμαρτιων προδε,δεξατο ινα ημας τους υποαμαρτιων προδε,10,100 Bodleian-Library-MS-Barocci-184_00040_fol-...,28


## Configuration


In [ ]:
batch_size = 64  # Batch size for training.
epochs = 1000  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.

# # Path to the data txt file on disk.
# data_path = "fra.txt"

## Prepare the data


In [ ]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
train_df["Proc_human"] = train_df['HUMAN_TRANSCRIPTION'].copy()

for i in range(len(train_df["Proc_human"])):

    train_df["Proc_human"][i] = "\t" + train_df["Proc_human"][i] + "\n"

    input_text = train_df["SYSTEM_TRANSCRIPTION"][i]
    target_text = train_df["Proc_human"][i]

    input_texts.append(input_text)
    target_texts.append(target_text)

    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
# # Vectorize the data.
# input_texts = []
# target_texts = []
# input_characters = set()
# target_characters = set()

# for line in lines[: min(num_samples, len(lines) - 1)]:
#     input_text, target_text, _ = line.split("\t")
#     # We use "tab" as the "start sequence" character
#     # for the targets, and "\n" as "end sequence" character.
#     target_text = "\t" + target_text + "\n"
#     input_texts.append(input_text)
#     target_texts.append(target_text)
#     for char in input_text:
#         if char not in input_characters:
#             input_characters.add(char)
#     for char in target_text:
#         if char not in target_characters:
#             target_characters.add(char)

In [ ]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 1875
Number of unique input tokens: 26
Number of unique output tokens: 28
Max sequence length for inputs: 83
Max sequence length for outputs: 87


In [ ]:
# input_characters = sorted(list(input_characters))
# target_characters = sorted(list(target_characters))
# num_encoder_tokens = len(input_characters)
# num_decoder_tokens = len(target_characters)
# max_encoder_seq_length = max([len(txt) for txt in input_texts])
# max_decoder_seq_length = max([len(txt) for txt in target_texts])

# print("Number of samples:", len(input_texts))
# print("Number of unique input tokens:", num_encoder_tokens)
# print("Number of unique output tokens:", num_decoder_tokens)
# print("Max sequence length for inputs:", max_encoder_seq_length)
# print("Max sequence length for outputs:", max_decoder_seq_length)

In [ ]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

## Build the model


In [ ]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)


## Train the model


In [ ]:
encoder_input_data.shape

(1875, 83, 26)

In [ ]:
encoder_input_data

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0.

In [ ]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
    callbacks=[early_stopping]
)
# Save model
model.save("s2s")


Epoch 1/1000
24/24 [==============================] - 9s 93ms/step - loss: 1.8267 - accuracy: 0.5876 - val_loss: 1.7614 - val_accuracy: 0.5548
Epoch 2/1000
24/24 [==============================] - 1s 36ms/step - loss: 1.5484 - accuracy: 0.6142 - val_loss: 1.6534 - val_accuracy: 0.5550
Epoch 3/1000
24/24 [==============================] - 1s 34ms/step - loss: 1.4051 - accuracy: 0.6176 - val_loss: 1.5800 - val_accuracy: 0.5612
Epoch 4/1000
24/24 [==============================] - 1s 36ms/step - loss: 1.3504 - accuracy: 0.6207 - val_loss: 1.5157 - val_accuracy: 0.5624
Epoch 5/1000
24/24 [==============================] - 1s 40ms/step - loss: 1.4258 - accuracy: 0.6195 - val_loss: 1.5100 - val_accuracy: 0.5645
Epoch 6/1000
24/24 [==============================] - 1s 33ms/step - loss: 1.2921 - accuracy: 0.6262 - val_loss: 1.4389 - val_accuracy: 0.5837
Epoch 7/1000
24/24 [==============================] - 1s 34ms/step - loss: 1.2525 - accuracy: 0.6383 - val_loss: 1.4133 - val_accuracy: 0.6059

WARNING  Found untraced functions such as lstm_cell_layer_call_fn,                           
         lstm_cell_layer_call_and_return_conditional_losses, lstm_cell_1_layer_call_fn,      
         lstm_cell_1_layer_call_and_return_conditional_losses while saving (showing 4 of 4). 
         These functions will not be directly callable after loading.

WARNING  <keras.layers.recurrent.LSTMCell object at 0x7f5c1e0f6590> has the same name        
         'LSTMCell' as a built-in Keras object. Consider renaming <class                     
         'keras.layers.recurrent.LSTMCell'> to avoid naming conflicts when loading with      
         `tf.keras.models.load_model`. If renaming is not possible, pass the object in the   
         `custom_objects` parameter of the load function.

WARNING  <keras.layers.recurrent.LSTMCell object at 0x7f5c10541290> has the same name        
         'LSTMCell' as a built-in Keras object. Consider renaming <class                     
         'keras.layers.recurrent.LSTMCell'> to avoid naming conflicts when loading with      
         `tf.keras.models.load_model`. If renaming is not possible, pass the object in the   
         `custom_objects` parameter of the load function.

## Run inference (sampling)

1. encode input and retrieve initial decoder state
2. run one step of decoder with this initial state
and a "start of sequence" token as target.
Output will be the next target token.
3. Repeat with the current target token and current states


In [ ]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence



You can now generate decoded sentences as such:


In [ ]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)


-
Input sentence: εγγενομεναπαδημησμεννωτες αλλατηε κλησει
Decoded sentence: και τους κακους δραν πανταχου κακως αει

-
Input sentence: του β ου του καλεαυτους πολλαγινεσθαι συγχωρ ον
Decoded sentence: τουτο το σχημα το υπερηφανον και ποληται

-
Input sentence: τες εμπυριζου σιμαμπελωνα αλλακαι οδξα
Decoded sentence: και τους κακους δραν πανταχου κακως αει

-
Input sentence: της εδιας πλσον εξιας πολλους εις την των αλ
Decoded sentence: τουτο το σχημα το υπερηφανον και ποπης

-
Input sentence: λοτρλων επιθυμιαν προκαλουμενος εμπυρι
Decoded sentence: πολλοις διαλανθανει και εις γνωσιν τοις

-
Input sentence: λει τον αμπελωνα συναπτειγαρ τοκαι καρπα
Decoded sentence: τουτο το σχημα το υπερηφανον και πεποιη

-
Input sentence: γη του πτωχουεντοις οικοιςχμων ωστεκακα
Decoded sentence: πολλοις διαλανθανει και εις τον του και τανταγγαντεν ταιαι

-
Input sentence: ικον κτησιν αποαρπαγης καιβωας συγεγοντεσ
Decoded sentence: τουτο το σχημα το υπερηφανον και πεποιη

-
Input sentence: αιτιαν του ε